<a href="https://colab.research.google.com/github/OSEUK/Sketch2Dream/blob/master/Sd_scribble_imageGeneration_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 본인 huggingface_hub 토큰으로 로그인
from huggingface_hub import login
login()

In [ ]:
# 1. 필요한 라이브러리 설치
!pip install diffusers transformers accelerate opencv-python safetensors
!pip install flask-ngrok flask pyngrok
!pip install flask-cors

In [ ]:
!pip install googletrans==4.0.0-rc1
!pip install boto3

In [ ]:
# 2. 모델 및 라이브러리 임포트
from flask import Flask, request, jsonify
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import cv2
from PIL import Image , ImageOps
import numpy as np
import io
from io import BytesIO
from pyngrok import ngrok
from flask_cors import CORS
import base64
from googletrans import Translator
import boto3
from werkzeug.utils import secure_filename
import uuid


# 모델 로드
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-scribble",
    torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16
)

# 5. GPU 사용 설정
pipe.to("cuda")

# 스케줄러 설정 (더 나은 품질을 위해)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# 번역기 설정
translator = Translator()




In [ ]:


# S3 버킷 목록 가져오기 테스트
try:
    response = s3.list_buckets()
    print("S3 연결 성공:", response['Buckets'])
except Exception as e:
    print("S3 연결 실패:", str(e))

In [17]:
# ngrok 터널 열기
ngrok.set_auth_token("")  # 이 부분에 실제 Ngrok 토큰을 추가하세요
ngrok_tunnel = ngrok.connect(5000)
public_url = ngrok_tunnel.public_url  # 정확한 URL만 추출
print("ngrok URL:", public_url)

# Flask 앱 설정
app = Flask(__name__)
# CORS 설정: 모든 경로에 대해 모든 출처에서의 요청 허용
CORS(app, resources={r"/*": {"origins": "*"}})

@app.route('/process_image', methods=['POST', 'OPTIONS'])
def process_image():
    # OPTIONS 요청에 대한 응답
    if request.method == 'OPTIONS':
        return jsonify({'status': 'Preflight check successful'}), 200


    # 텍스트 데이터와 파일 수신
    input_text = request.form.get('text')
    image_file = request.files['image']

    # 한글을 영어로 번역 (translator 사용 가정)
    input_text_english = translator.translate(input_text, src='ko', dest='en').text
    print(input_text_english)

    # 이미지 처리 (PIL을 사용하여 열기)
    image = Image.open(image_file)

    # 이미지가 RGBA 또는 다른 모드일 수 있으므로 RGB로 변환합니다.
    image = image.convert('RGB')

    # 이미지를 반전시킵니다.
    image = ImageOps.invert(image)


    image.save("/content/input.png")
    # 프롬프트 설정
    prompt = (
        f"Generate a high-resolution, realistic photograph of {input_text_english}. "
        "The image should feature realistic lighting, sharp details, and vivid backgrounds."
    )
    negative_prompt = "low quality, blurry, oversaturated, worst quality"


    # 이미지 변환
    with torch.no_grad():
        result = pipe(
            prompt=prompt,
            image=image,
            negative_prompt = negative_prompt,
            num_inference_steps=30,
            guidance_scale=7.5,
            controlnet_conditioning_scale=1.0
          )

    # 변환된 결과 이미지

    result_image = result.images[0]
    result_image.save("/content/output.png")
    # 결과 이미지를 메모리로 저장
    buffered = BytesIO()
    result_image.save(buffered, format="PNG")
    buffered.seek(0)

          # S3에 이미지 업로드 (파일명은 유니크하게 설정)
    filename = secure_filename(f"generated_image_{uuid.uuid4()}.png")
    s3.upload_fileobj(
      buffered,  # 이미지 파일
      BUCKET_NAME,  # S3 버킷 이름
      filename,  # 파일 이름
      ExtraArgs={'ContentType': 'image/png'}  # ContentType만 설정
    )


        # S3 URL 생성
    image_url = f"https://{BUCKET_NAME}.s3.amazonaws.com/{filename}"

    return jsonify({'image_url': image_url, 'text': input_text})

    # except Exception as e:
    #     return jsonify({'error': 'Failed to process image', 'message': str(e)}), 400


if __name__ == '__main__':
    app.run()

ngrok URL: https://d844-34-124-218-54.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


cup of coffee


  0%|          | 0/30 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [27/Sep/2024 12:44:09] "POST /process_image HTTP/1.1" 200 -


In [ ]:
# 계속하다가 뜬금없는 에러가 뜬다면 한번쯤 해보기
!kill $(pgrep ngrok)